# Agent-Based Model developed by Ejsmond 2018

Port of the simulation of [Ejsmond 2018](https://www.nature.com/articles/s41467-018-06821-x) from matlab to python. Original code: [https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-018-06821-x/MediaObjects/41467_2018_6821_MOESM4_ESM.txt](https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-018-06821-x/MediaObjects/41467_2018_6821_MOESM4_ESM.txt).

In [1]:
import numpy as np
import pandas as pd

## Initialize Global Model Variables

In [354]:
rng = np.random.default_rng(); # initialize RNG

N = 1000  # population size
n_gen = 10600  # generation number
epitope_space_size = 1000 # "epitope" space size (1000*1000)

red_queen = True # toggles co-evolution of host-parasite -> 

dcl_ST = True # toggle ST analysis - only plotting !not clear!
isv2 = 0 # indices for saving data on pathogens and hosts !not clear!

start_save = 2000  # start of saving data on host and pathogens
g4sav1 = range(start_save, n_gen + 1, 10)  # plotting of ST and haplotypes


## Initialize Host Data

In [240]:
host_mu = 0.1 # host mutation rate
# possible step directions a host can take when mutating
host_mu_steps = np.matrix([
    [-1, -1, -1,  0, 0, 0,  1, 1, 1],
    [-1,  0,  1, -1, 0, 1, -1, 0, 1]
]) 

host_fitness_reproduction = 1.25 # fitness of reproducing hosts
host_fitness_birth = 0.25 # fitness of hosts at birth

# initialize positions of allele one in epitope space (x,y)
host_allele_1_start = rng.integers(low=1, high=epitope_space_size, size=(2, N)) 
# initialize positions of allele two in epitope space (x,y)
host_allele_2_start = rng.integers(low=1, high=epitope_space_size, size=(2, N))

host_allele_1 = host_allele_1_start
host_allele_2 = host_allele_2_start

# later used, defined here for global scope
# effective number of alleles, subsample from 100 !not entirely clear!
host_allele_count_effective = np.zeros((len(g4sav1))) 

# record values over all the generations of the simulation (vector length == number of generations)
# number of dying hosts from each generation
hist_host_dying = np.zeros((n_gen))
# number of hosts not reproducing
hist_host_not_reproducing = np.zeros((n_gen))

# initial fitness for all hosts
host_fitness = np.zeros((N)) + host_fitness_birth
# tally for host brood
host_brood_count = np.zeros((N))
host_age = np.zeros((N))

# Supertype start positions (x,y) !not clear!
STs_1_2_start = np.matrix([range(1,N+1), range(N+1, 2*N+1)])


## Initialize Parasite Data

In [432]:
pat_count = 10  # number of pathogen species
pat_mu = 1
pat_mu_steps = np.matrix([-1, 0, 1])
pat_added = 0.01  # fraction of pathogens from initial population that is added every generation to the pool from which recruits are drawn

# pathogen haplotype, as x,y coordinate in the epitope space
pat_haplotype_start = rng.integers(low=1, high=epitope_space_size, size=(2, N, pat_count))

# record pathogen properties in g4sav1 increments
g4hist_pat_haplotype_count = np.zeros((pat_count,len(g4sav1)))
g4hist_pat_haplotype = np.zeros((N,len(g4sav1),2))
